In [1]:
%cd ../src
%load_ext autoreload
%autoreload 2

/home/ubuntu/SPVD_Lightning/src


In [2]:
from models.ddpm_unet_cattn import SPVUnet
import torch
import lightning as L
from models.g_spvd import GSPVD

In [3]:
from dataloaders.shapenet.shapenet_loader import ShapeNet

categories = ['car']
path = "../data/ShapeNet"

te = ShapeNet(path, "test", 2048, categories, load_renders=True)

Loading (test) renders for car (02958343):   0%|          | 0/704 [00:00<?, ?it/s]

In [4]:
from utils.helper_functions import normalize_to_unit_sphere
from metrics.chamfer_dist import ChamferDistanceL2
from metrics.PyTorchEMD.emd import earth_mover_distance as EMD

cd = ChamferDistanceL2()
point_clouds = normalize_to_unit_sphere(torch.stack([te[i]['pc'] for i in range(len(te))])).cuda()
P = torch.randn(2048, 3, requires_grad=True, device='cuda')

optim = torch.optim.Adam([P], lr=0.05)

for _ in range(150):
    loss = sum(cd(P.unsqueeze(0), pc.unsqueeze(0)) for pc in point_clouds) / len(point_clouds)
    optim.zero_grad()
    loss.backward()
    optim.step()
    if _ % 10 == 0:
        print(f"Iteration {_}, Loss: {loss.item()}")
print("Optimized Point Cloud:")

Iteration 0, Loss: 1040.35986328125
Iteration 10, Loss: 378.63958740234375
Iteration 20, Loss: 133.8667449951172
Iteration 30, Loss: 49.01374816894531
Iteration 40, Loss: 20.034223556518555
Iteration 50, Loss: 9.900450706481934
Iteration 60, Loss: 6.1638689041137695
Iteration 70, Loss: 4.7814130783081055
Iteration 80, Loss: 4.3000168800354
Iteration 90, Loss: 4.145587921142578
Iteration 100, Loss: 4.092180252075195


KeyboardInterrupt: 

In [ ]:
from utils.visualization import display_pointclouds_grid
import numpy as np
P = 5 * P
display_pointclouds_grid(np.array([P.detach().cpu()]), point_size=0.3)

Output()